In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

import altair as alt
from altair.vega import v5
from IPython.display import HTML
from sklearn import preprocessing
from tqdm.notebook import tqdm
import lightgbm as lgb
import numpy as np
import pandas as pd
import lightgbm as lgb
import glob
import os
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
from tqdm.notebook import tqdm
import optuna

import gc, datetime, random

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from collections import defaultdict, Counter

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings("ignore")

# helper funcs

In [ ]:

def reduce_mem(df, verbose=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def reduce_mem_usage(df, verbose=False):
    return reduce_mem(df, verbose=verbose)


In [ ]:
def preprocess1(df, train_flag):
#     cat_cols = cat_cols
    cat_cols = ['City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat']
    cate_cols = cat_cols
    feature_key = cat_cols
    feature_target = cat_cols
    for f in tqdm(cate_cols):
        map_dict = dict(zip(df[f].unique(), range(df[f].nunique())))
        df[f + '_count'] = df[f].map(df[f].value_counts())
    df = reduce_mem(df)
    ##########################groupby feature#######################
    def group_fea(df,key,target):
        tmp = df.groupby(key, as_index=False)[target].agg({
            key+'_'+target + '_nunique': 'nunique',
        }).reset_index()
        del tmp['index']
        return tmp

    for key in tqdm(feature_key):
        for target in feature_target:
            tmp = group_fea(df,key,target)
            df = df.merge(tmp,on=key,how='left')
            
            
    #统计做了groupby特征的特征
    group_list = []
    for s in df.columns:
        if '_nunique' in s:
            group_list.append(s)
    #print(group_list)
    return df, group_list

def preprocess(train_df, test_df, num_clusters=10):

    cat_cols = ['City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat']
    enc_list=[]
    
    
    
#     test_df, _ = preprocess1(test_df, 0)
#     train_df, group_list = preprocess1(train_df.copy(), 1)
    
#     skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=2020)
#     enc_list = group_list + cat_cols
    
#     for f in tqdm(enc_list):
#         if len(train_df[f].unique()) > len(train_df)//100:
#             print("f ", f)
#             continue
#         train_df[f + '_target_enc'] = 0
#         test_df[f + '_target_enc'] = 0
#         for i, (trn_idx, val_idx) in enumerate(skf.split(train_df, train_df['label'])):
#             trn_x = train_df[[f, 'label']].iloc[trn_idx].reset_index(drop=True)
#             val_x = train_df[[f]].iloc[val_idx].reset_index(drop=True)
#             enc_df = trn_x.groupby(f, as_index=False)['label'].agg({f + '_target_enc': 'mean'})
#             val_x = val_x.merge(enc_df, on=f, how='left')
#             test_x = test_df[[f]].merge(enc_df, on=f, how='left')
#             val_x[f + '_target_enc'] = val_x[f + '_target_enc'].fillna(train_df['label'].mean())
#             test_x[f + '_target_enc'] = test_x[f + '_target_enc'].fillna(train_df['label'].mean())
#             train_df.loc[val_idx, f + '_target_enc'] = val_x[f + '_target_enc'].values
#             test_df[f + '_target_enc'] += test_x[f + '_target_enc'].values / skf.n_splits
#             gc.collect()
#         gc.collect()
        
    return train_df.reset_index(drop=True), test_df.reset_index(drop=True), enc_list


In [ ]:
train = pd.read_csv("/kaggle/input/job-a-thon-rw/train_Df64byy.csv")
test = pd.read_csv("/kaggle/input/job-a-thon-rw/test_YCcRUnU.csv")
sub = pd.read_csv("/kaggle/input/job-a-thon-rw/sample_submission_QrCyCoT.csv")
sub['ID'] = test['ID']


train = train.fillna(-1)
test = test.fillna(-1)


train = train.drop(['ID'], axis=1)
test = test.drop(['ID'], axis=1)

train.head()

In [ ]:
nn = [
# 'ID',
 'City_Code',
 'Region_Code',
 'Accomodation_Type',
 'Reco_Insurance_Type',
 'Upper_Age',
 'Lower_Age',
 'Is_Spouse',
 'Health Indicator',
 'Holding_Policy_Duration',
 'Holding_Policy_Type',
 'Reco_Policy_Cat',
 'Reco_Policy_Premium',
 'label']

target = 'label'
train.columns = nn


In [ ]:
cat_cols = ['City_Code',
 'Region_Code',
 'Accomodation_Type',
 'Reco_Insurance_Type',
 'Is_Spouse',
 'Health Indicator',
 'Holding_Policy_Duration',
 'Holding_Policy_Type',
 'Reco_Policy_Cat']



In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc.fit(np.array(train['Reco_Policy_Cat']).reshape(-1, 1))
df_tr = pd.DataFrame(enc.transform(np.array(train['Reco_Policy_Cat']).reshape(-1, 1)))
df_tes = pd.DataFrame(enc.transform(np.array(test['Reco_Policy_Cat']).reshape(-1, 1)))
tr_col = []
for i in df_tr.columns:
    tr_col.append("OH_" + str(i))
df_tr.columns = tr_col

tr_col = []
for i in df_tes.columns:
    tr_col.append("OH_" + str(i))
df_tes.columns = tr_col


train = pd.concat([train, df_tr], axis=1)
test = pd.concat([test, df_tes], axis=1)
train.head()

In [ ]:
i='Region_Code'
xx = train[target].value_counts()[0] / train[target].value_counts()[1] 
df = pd.DataFrame({0 : train[train[target]==0][i].value_counts()/xx, 1: train[train[target]==1][i].value_counts()})
plt.plot(np.abs((df[0]-df[1] )/df.min(axis=1))*100)

idx = list((np.abs((df[0]-df[1] )/df.min(axis=1))*100).sort_values(ascending=False)[:200].index)

train['Region_Code_limited'] = train['Region_Code']
train['Region_Code_limited']
for j in train['Region_Code_limited'].index:
    if train['Region_Code_limited'][j] in idx:
        continue
    else :
        train['Region_Code_limited'][j] = -1
        
        
test['Region_Code_limited'] = test['Region_Code']
test['Region_Code_limited']
for j in test['Region_Code_limited'].index:
    if test['Region_Code_limited'][j] in idx:
        continue
    else :
        test['Region_Code_limited'][j] = -1
        
        
cat_cols.append('Region_Code_limited')
cat_cols

In [ ]:
# train['Region_Code_limited'].value_counts()

# balaced check

In [ ]:
# train['Response'].value_counts()

In [ ]:
# train1 = train[train['Response'] == 1]
# train = pd.concat([train, train1], axis=0).reset_index(drop=True)
# train = pd.concat([train, train1], axis=0).reset_index(drop=True)
# 

In [ ]:
# train['Response'].value_counts()

In [ ]:
# for i in train.columns:
#     print(i,  len(train[i].unique()))

In [ ]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
train.head()

In [ ]:
X_train, X_validation = train_test_split(train, train_size=0.80000, random_state=42)
X_train.head()

In [ ]:
X_train, X_validation, enc_list = preprocess(X_train.copy(), X_validation.copy(), num_clusters=10)

In [ ]:
y_validation = X_validation[target]
y_train = X_train[target]

X_train = X_train.drop(target, axis=1)
X_validation = X_validation.drop(target, axis=1)
X_validation.head()

In [ ]:
train_new, test, enc_list = preprocess(train.copy(), test.copy(), num_clusters=10)

# Tuning

In [ ]:
from sklearn.metrics import roc_auc_score

# CAtbOost

In [ ]:
# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    param = {
        "eval_metric": "AUC",
        'logging_level': 'Silent',
        'boosting_type' : 'Plain' ,
        "learning_rate": trial.suggest_float("learning_rate", 1e-2, 0.5, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-8, 500.0, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 12, log=False),
        'task_type':"GPU",       
        "iterations": trial.suggest_int("iterations", 50, 5000),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 1e-2, 0.5, log=True),
        "max_ctr_complexity": trial.suggest_int("max_ctr_complexity", 2, 2),
        "max_bin": trial.suggest_int("max_bin", 32, 200),
        "use_best_model" : False,
    }
    
    
    model = CatBoostClassifier(**param)
    model = model.fit(
                        X_train, y_train, 
                        cat_features=cat_cols,
                        use_best_model=False,
#                         eval_set=(X_validation, y_validation),
                     )
    preds = model.predict_proba(X_validation)[:, 1]
    accuracy = sklearn.metrics.roc_auc_score(y_validation, preds)
    return accuracy


In [ ]:
params = {'learning_rate': 0.0769396280700289,
 'l2_leaf_reg': 117.9397659497423,
 'max_depth': 5,
 'iterations': 3465,
 'bagging_temperature': 0.023880907815556483,
 'max_ctr_complexity': 2,
 'max_bin': 154}

In [ ]:
# # study = optuna.create_study(direction="minimize")
# study = optuna.create_study(direction="maximize")
# # increase n_trails to test more params
# study.optimize(objective, n_trials=60)

# print("Number of finished trials: {}".format(len(study.trials)))
# trial = study.best_trial
# print("Best trial:")
# # paramss.append(trial.params)

# print("  Value: {}".format(trial.value))

# print("  Params: ")


# for key, value in trial.params.items():
#     print("    {}: {}".format(key, value))
    

# params = trial.params
# trial.params


In [ ]:

params['eval_metric'] = 'AUC'
params['logging_level'] = 'Silent'
params['boosting_type'] = 'Plain'
# params['task_type'] = "GPU"


model = CatBoostClassifier(**params)
model.get_params()

In [ ]:
params_list = [
#     {'learning_rate': 0.03504690553120952, 'l2_leaf_reg': 16.674632013331745, 'max_depth': 9, 'iterations': 67, 'bagging_temperature': 0.018863676053720923, 'max_ctr_complexity': 2, 'max_bin': 190},
#     {'learning_rate': 0.4976951327091617, 'l2_leaf_reg': 0.06382353714637816, 'max_depth': 4, 'iterations': 3288, 'bagging_temperature': 0.012422864451370747, 'max_ctr_complexity': 2, 'max_bin': 38},
#     {'learning_rate': 0.03417560009829608, 'l2_leaf_reg': 0.04325250648764644, 'max_depth': 9, 'iterations': 4064, 'bagging_temperature': 0.012711310396452059, 'max_ctr_complexity': 2, 'max_bin': 139},
#     {'learning_rate': 0.017217004608133267, 'l2_leaf_reg': 2.490993431649518e-06, 'max_depth': 12, 'iterations': 1513, 'bagging_temperature': 0.13830457550750483, 'max_ctr_complexity': 2, 'max_bin': 71},
    
    # last col removed 
    {'learning_rate': 0.1197155786554287, 'l2_leaf_reg': 1.792618447708568, 'max_depth': 3, 'iterations': 3979, 'bagging_temperature': 0.03598742456824714, 'max_ctr_complexity': 2, 'max_bin': 125},
    {'learning_rate': 0.03544766725209928, 'l2_leaf_reg': 1.370803084483364, 'max_depth': 3, 'iterations': 3708, 'bagging_temperature': 0.03901618689549485, 'max_ctr_complexity': 2, 'max_bin': 124},
    {'learning_rate': 0.03436720800438295, 'l2_leaf_reg': 0.09215163878772453, 'max_depth': 3, 'iterations': 3870, 'bagging_temperature': 0.020452925772898055, 'max_ctr_complexity': 2, 'max_bin': 88},
    {'learning_rate': 0.1266276143771968, 'l2_leaf_reg': 2.0110845275197993, 'max_depth': 2, 'iterations': 782, 'bagging_temperature': 0.029945781198908217, 'max_ctr_complexity': 2, 'max_bin': 157},
    {'learning_rate': 0.04104228366631473, 'l2_leaf_reg': 5.826887261802294, 'max_depth': 6, 'iterations': 2649, 'bagging_temperature': 0.06985674308863894, 'max_ctr_complexity': 2, 'max_bin': 173},
    
    
    
    {'learning_rate': 0.0769396280700289, 'l2_leaf_reg': 117.9397659497423, 'max_depth': 5, 'iterations': 3465, 'bagging_temperature': 0.023880907815556483, 'max_ctr_complexity': 2, 'max_bin': 154},
    {'learning_rate': 0.12059119126628469, 'l2_leaf_reg': 0.1974485737651515, 'max_depth': 3, 'iterations': 3574, 'bagging_temperature': 0.012096758886745147, 'max_ctr_complexity': 2, 'max_bin': 66},
    {'learning_rate': 0.06623001793892062, 'l2_leaf_reg': 28.646322073736556, 'max_depth': 3, 'iterations': 3542, 'bagging_temperature': 0.012274728415153181, 'max_ctr_complexity': 2, 'max_bin': 59},
    {'learning_rate': 0.10277504026045622, 'l2_leaf_reg': 6.093172008641855, 'max_depth': 2, 'iterations': 3339, 'bagging_temperature': 0.36483302461032735, 'max_ctr_complexity': 2, 'max_bin': 44},
    {'learning_rate': 0.016976100171073007, 'l2_leaf_reg': 1.8060489262789463, 'max_depth': 4, 'iterations': 4134, 'bagging_temperature': 0.4785455551994752, 'max_ctr_complexity': 2, 'max_bin': 94},
    {'learning_rate': 0.039148597638397585, 'l2_leaf_reg': 22.14201754863478, 'max_depth': 3, 'iterations': 3915, 'bagging_temperature': 0.07737592690958507, 'max_ctr_complexity': 2, 'max_bin': 77},
    
    {'learning_rate': 0.028883448110346648, 'l2_leaf_reg': 482.9056272005946, 'max_depth': 5, 'iterations': 4291, 'bagging_temperature': 0.02350408621458312, 'max_ctr_complexity': 2, 'max_bin': 54},
    {'learning_rate': 0.02260331189053014, 'l2_leaf_reg': 101.03612749341279, 'max_depth': 2, 'iterations': 3026, 'bagging_temperature': 0.03082718141487049, 'max_ctr_complexity': 2, 'max_bin': 60},
    {'learning_rate': 0.028391019999274084, 'l2_leaf_reg': 1.6714091660819037, 'max_depth': 4, 'iterations': 3588, 'bagging_temperature': 0.021733070715488616, 'max_ctr_complexity': 2, 'max_bin': 54},
    {'learning_rate': 0.055396516759364335, 'l2_leaf_reg': 0.7207341110153501, 'max_depth': 4, 'iterations': 3877, 'bagging_temperature': 0.01649717469060937, 'max_ctr_complexity': 2, 'max_bin': 34},
    {'learning_rate': 0.10782034981327523, 'l2_leaf_reg': 36.11128296358572, 'max_depth': 6, 'iterations': 2273, 'bagging_temperature': 0.016578649076498257, 'max_ctr_complexity': 2, 'max_bin': 112},
]

#last 10 diff params : non bal less col versio 9

In [ ]:
%%time
no_of_stacks = 1
stack_df = pd.DataFrame(index = train.index)
stack_df[target] = train[target]


stack_df_test = pd.DataFrame(index = test.index)
# stack_df_test[target] = train[target]



for no_of_stack in tqdm(range(no_of_stacks)):
    stack_df[no_of_stack] = 0.0
    params = params_list[no_of_stack]
    
    params = {'learning_rate': 0.03135940484917028, 'l2_leaf_reg': 301.1739543936941, 'max_depth': 8, 'iterations': 3682, 'bagging_temperature': 0.14945648119943328, 'max_ctr_complexity': 2, 'max_bin': 54}

    params['eval_metric'] = 'AUC'
    params['logging_level'] = 'Silent'
    params['boosting_type'] = 'Plain'
#     params['task_type'] = "GPU"

    model = CatBoostClassifier(**params)
    model.get_params()
    
    
    
    n_folds = 20
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    y_pred = np.array([0.0] * len(test))
    
    for i, (trn_idx, val_idx) in enumerate(skf.split(train, train[target])):
        X_train = train.loc[trn_idx, :].drop(target, axis=1)#.reset_index(drop=True)
        X_validation = train.loc[val_idx, :].drop(target, axis=1)
        y_train = train.loc[trn_idx, target]
        y_validation = train.loc[val_idx, target]

        X_train, X_validation, enc_list = preprocess(X_train.copy(), X_validation.copy(), num_clusters=10)
        #test is already preprocessed.
        
        X_train_t, X_valid_t, y_train_t, y_valid_t = train_test_split(X_train, y_train, train_size=0.80000, random_state=42)

        
        #choose eval_Set or not to go with eval set
        model.fit(
            X_train, y_train,
            cat_features=cat_cols,
#             eval_set=(X_valid_t, y_valid_t),
            plot=True
        )
        
        pred_X_validation = model.predict_proba(X_validation)[:, 1]
        stack_df.loc[val_idx, no_of_stack] = pred_X_validation
        
        print(no_of_stack, i, sklearn.metrics.roc_auc_score(y_validation, model.predict_proba(X_validation)[:, 1]))

        y_pred += model.predict_proba(test)[:, 1]/(n_folds)
        gc.collect()
        
    stack_df_test[no_of_stack] = y_pred

In [ ]:
# n_folds = 10
# skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# y_pred = np.array([0.0] * len(test))

# for i, (trn_idx, val_idx) in enumerate(skf.split(train, train[target])):
#     X_train = train.loc[trn_idx, :].drop(target, axis=1)#.reset_index(drop=True)
#     X_validation = train.loc[val_idx, :].drop(target, axis=1)
#     y_train = train.loc[trn_idx, target]
#     y_validation = train.loc[val_idx, target]
    
    
#     X_train, X_validation, enc_list = preprocess(X_train.copy(), X_validation.copy(), num_clusters=10)
#     #test is already preprocessed.
    
        
#     model.fit(
#     X_train, y_train,
#     cat_features=cat_cols,
#     eval_set=(X_validation, y_validation),
#     plot=True
#     )
    
#     y_pred += model.predict_proba(test)[:, 1]/n_folds

    
#     gc.collect()
    

In [ ]:
# n_folds = 5
# skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# y_pred = np.array([0.0] * len(test))

# for i, (trn_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
#     X_train_t = X_train.loc[trn_idx, :]#.drop(target, axis=1)#.reset_index(drop=True)
#     X_valid_t = X_train.loc[val_idx, :]#.drop(target, axis=1)
#     y_train_t = y_train.loc[trn_idx]
#     y_valid_t = y_train.loc[val_idx]
    
    
#     model.fit(
#         X_train_t, y_train_t,
#         cat_features=cat_cols,
#         eval_set=(X_valid_t, y_valid_t),
#         plot=True
#     )
    
#     print(i, sklearn.metrics.roc_auc_score(y_validation, model.predict_proba(X_validation)[:, 1]))
    
#     y_pred += model.predict_proba(test)[:, 1]/n_folds

    
#     gc.collect()
    

In [ ]:
# # X_train, X_validation, y_train, y_validation, test, enc_list = preprocess(df, train_flag=1)
# model.fit(
#     X_train, y_train,
#     cat_features=cat_cols,
#     eval_set=(X_validation, y_validation),
#     plot=True
# )

# y_pred = model.predict_proba(test)[:, 1]
# gc.collect()

In [ ]:
# # y_pred = model.predict_proba(test_df)[:, 1]
# sub['Response'] = y_pred
# sub.to_csv('sub_job_a_thon.csv', index=False)
# sub.head()

In [ ]:
from IPython.display import FileLinks
FileLinks('.')

# LGBM

In [ ]:
daa = pd.concat([X_train, X_validation, test], axis=0)
# daa
for i  in  ['City_Code', 'Accomodation_Type', 'Reco_Insurance_Type', 'Is_Spouse', 'Health Indicator', 'Holding_Policy_Duration']:
    le = LabelEncoder()
#     print(daa[i][0].dtype)

    daa[i] = le.fit(daa[i].astype(str))
    X_train[i] = le.transform(X_train[i].astype(str))
    X_validation[i] = le.transform(X_validation[i].astype(str))
    test[i] = le.transform(test[i].astype(str))

cat_col_lgb = []
for e, i in enumerate(X_train.columns):
    if i in cat_cols:
        cat_col_lgb.append(e)
cat_col_lgb


In [ ]:
# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    #data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
#     file = "../input/indoornavigationandlocationsum-xyz/train/5a0546857ecc773753327266_1000_train.csv"
    file = train_files[i]
    data = pd.read_csv(file, )
    print(i)
    
    x_train = data.drop(['x', 'y', 'f', 'path'], axis=1)
    le = LabelEncoder()
    data['f'] = le.fit_transform(data['f'])
    y_trainy = data.loc[:,'f']
    
    train_x, valid_x, train_y, valid_y = train_test_split(x_train, y_trainy, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)
#     print(valid_y.value_counts())
#     print(train_y.value_counts())
    
    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "num_class" : len(data.loc[:,'f'].unique()),
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    

    
    
    
    
    
    
    
    gbm = lgb.train(param, dtrain)
    preds = gbm.predict_proba(valid_x)[:, 1]
    pred_labels = (preds)
    accuracy = np.sqrt(sklearn.metrics.roc_auc_score(valid_y, pred_labels))
#     accuracy = np.sqrt(sklearn.metrics.accuracy_score(valid_y, pred_labels))
    
    return accuracy

In [ ]:
# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    
    params = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 50, 5000),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        'cat_feature':cat_col_lgb, 
        'verbose':-1,
        
    }
    
#     model = CatBoostClassifier(**param)
#     model = model.fit(
#                         X_train, y_train, 
#                         cat_features=cat_cols,
#                         use_best_model=False,
# #                         eval_set=(X_validation, y_validation),
#                      )
    
    
    model = lgb.LGBMClassifier(**params)
    model.fit(
        X_train, y_train,
        verbose=-1,
        
    )
    preds = model.predict_proba(X_validation)[:, 1]
    accuracy = sklearn.metrics.roc_auc_score(y_validation, preds)
    return accuracy


In [ ]:
params = {'learning_rate': 0.02039671552078551, 'l2_leaf_reg': 0.9538065032003882, 'max_depth': 8, 'iterations': 1500, 'bagging_temperature': 0.29925486960824327, 'max_bin': 38}


In [ ]:
# study = optuna.create_study(direction="minimize")
study = optuna.create_study(direction="maximize")
# increase n_trails to test more params
study.optimize(objective, n_trials=30)

print("Number of finished trials: {}".format(len(study.trials)))
trial = study.best_trial
print("Best trial:")
# paramss.append(trial.params)

print("  Value: {}".format(trial.value))

print("  Params: ")


for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
    
params = trial.params
trial.params

In [ ]:
params['metric'] = 'auc'
params['verbose'] = -1
params['boosting_type'] = 'gbdt'
params['cat_feature'] = cat_col_lgb


model = lgb.LGBMClassifier(**params)


In [ ]:

# n_folds = 5
# skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# y_pred = np.array([0.0] * len(test))

# for i, (trn_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
#     X_train_t = X_train.loc[trn_idx, :]#.drop(target, axis=1)#.reset_index(drop=True)
#     X_valid_t = X_train.loc[val_idx, :]#.drop(target, axis=1)
#     y_train_t = y_train.loc[trn_idx]
#     y_valid_t = y_train.loc[val_idx]
    
#     model.fit(
#         X_train_t, y_train_t,
#         eval_set=(X_valid_t, y_valid_t),
#         verbose=-1,       
#     )
    
#     print(i, sklearn.metrics.roc_auc_score(y_validation, model.predict_proba(X_validation)[:, 1]))
#     y_pred += model.predict_proba(test)[:, 1]/n_folds

    
#     gc.collect()
    

In [ ]:
# model.fit(
#     X_train, y_train,
#     eval_set=(X_validation, y_validation),
#     verbose=-1,
# )
# y_pred = model.predict_proba(test)[:, 1]

# tpot

In [ ]:
from tpot import TPOTClassifier
tp = TPOTClassifier()
tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, random_state=42, scoring='roc_auc')
tpot.fit(X_train, y_train)
print(tpot.score(X_validation, y_validation))
tpot.export('tpot_iris_pipeline.py')

In [ ]:
tpot.export('tpot_iris_pipeline.py')

# submission

In [ ]:
# y_pred = model.predict_proba(test_df)[:, 1]
sub['Response'] = y_pred
sub.to_csv('sub_job_a_thon.csv', index=False)
sub.head()

In [ ]:
from IPython.display import FileLinks
FileLinks('.')